# 11-HUBO: Higher order unconstraint binary optimization

If you want to handle higher order model as follows:

$$
H=\sum_{i} h_{i} \sigma_{i}+\sum_{i<j} J_{i j} \sigma_{i} \sigma_{j}+\sum_{i, j, k} K_{i, j, k} \sigma_{i} \sigma_{j} \sigma_{k} \cdots 
\\
\sigma_i \in \{-1, 1\}, i=1,\cdots N
$$

use .sample_hubo

HUBO: Higher order unconstraint binary optimization

In [24]:
# https://openjij.github.io/OpenJij_Documentation/build/html/5-HigherOrderModel.html

import openjij as oj

# Only SASampler can handle HUBO.
sampler = oj.SASampler()

# define Jij
J = {(0,): -1, (0,1): -1, (0,1,2): 1}

#  make HUBO
response = sampler.sample_hubo(J, vartype="SPIN")
print(response)

   0  1  2 energy num_oc.
0 +1 +1 -1   -3.0       1
['SPIN', 1 rows, 1 samples, 3 variables]


In [25]:
# 辞書のkeyは数値以外も扱うことができる。
# define Jij
J = {('a',): -1, ('a','b'): -1, ('a','b','c'): 1}

#  make HUBO
response = sampler.sample_hubo(J, vartype="SPIN")
print(response)

   a  b  c energy num_oc.
0 +1 +1 -1   -3.0       1
['SPIN', 1 rows, 1 samples, 3 variables]
